# Explainable reasoning with ChiRho (categorical variables)


The **Explainable Reasoning with ChiRho** package aims to provide a systematic, unified approach to causal explanation computations in terms of different probabilistic queries over expanded causal models that are constructed from a single generic program transformation applied to an arbitrary causal model represented as a ChiRho program. The approach of reducing causal queries to probabilistic computations on transformed causal models is the foundational idea behind all of ChiRho. The key strategy underlying "causal explanation" queries is their use of auxiliary variables representing uncertainty over which interventions or preemptions to apply, implicitly inducing a search space over counterfactuals.

The goal of this notebook is to illustrate how the package can be used to provide approximate method of answering a range of causal explanation queries with respect to models in which the key role is played by categorical variables. Continuous variables are in the focus of another notebook.

In [another notebook](https://basisresearch.github.io/chirho/actual_causality.html) we illustrate how the module allows for a faithful reconstration of a specific notion of local explanation that inspired some of the conceptual moves underlying the current implementation  (the so-called Halpern-Pearl modified definition of actual causality [(J. Halpern, MIT Press, 2016)](https://mitpress.mit.edu/9780262537131/actual-causality/)).

**Outline**

[Introduction and motivations](#intuitions-and-motivations)
    
- [The but-for condition](#the-but-for-condtition)

- [Witness nodes and context-sensitivity](#witness-nodes-and-context-sensitivity)

[Simplified actual causality](#simplified-actual-causality)

[Probability of causation](#probability-of-causation)

[Causal explanation](#causal-explanation)

[Responsibility attribution](#responsibility-attribution)


In [1]:
import os

import pyro
import pyro.distributions as dist
import torch

from chirho.observational.handlers import condition
from chirho.counterfactual.handlers.counterfactual import MultiWorldCounterfactual
from chirho.explainable.handlers import SearchForExplanation
                                            
from chirho.indexed.ops import (IndexSet, gather, indices_of) 
from chirho.interventional.handlers import do


smoke_test = ('CI' in os.environ)
runs_n = 5 if smoke_test else 8000

## Introduction and motivations

Let's start with a model of a very simplistic situation, in which a forest fire can be cause by exactly one of two causes: a match being dropped (`match_dropped`), or a lightning (`lightning`), and either of these factors on its own is already deterministcally sufficient for the `forest_fire` to occur. In general, you think a match being dropped is more likely than a lightning (we use fairly large probabilities for the same of example transparency). 

In [15]:
def ff_disjunctive():
        match_dropped = pyro.sample("match_dropped", dist.Bernoulli(0.7)) # notice uneven probs here
        lightning = pyro.sample("lightning", dist.Bernoulli(0.4))

        forest_fire = pyro.deterministic("forest_fire", torch.max(match_dropped, lightning), event_dim=0)

        return {"match_dropped": match_dropped, "lightning": lightning,
            "forest_fire": forest_fire}

# each run is stochastic
ff_disjunctive()

{'match_dropped': tensor(1.),
 'lightning': tensor(0.),
 'forest_fire': tensor(1.)}

Suppose in this particular case you know a forest fire has occured, a match has been dropped, but no lightning occured. This further assumption can be introduced by conditioning on these observations:

In [17]:
observations = {"match_dropped": torch.tensor(1.), 
                "lightning": torch.tensor(0.),
                "forest_fire": torch.tensor(1.)}

with condition(data = observations):
    with pyro.poutine.trace() as tr:
        ff_disjunctive()

# now it is determined how things play out
print({key: tr.trace.nodes[key]["value"] for key in tr.trace.nodes.keys()})


{'match_dropped': tensor(1.), 'lightning': tensor(0.), 'forest_fire': tensor(1.)}


In a particular context like this, when you know what happened, or at least know the values of some of the variables at play, you might be interested in using the model to answer a range of causal-explanation related questions.

- Did the match being dropped actually cause the fire?
- Suppose you only know that the forest fire occured, what are the likely explanations? How likely are they?
- Suppose you know both factors occured, to what extent should they be deemed responsible for this outcome?

Let's see how these can be addressed using ChiRho. 

### The but-for condition

### Witness nodes and context-sensitivity

## Simplified actual causality

## Probability of causation

## Causal explanation

## Responsibility attribution
